In [61]:
import numpy as np
import sys
from scipy import sparse
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import networkx as nx
from sklearn.preprocessing import StandardScaler
%matplotlib inline
import scipy as sci
from sklearn.cluster import KMeans
import sklearn.metrics as sm

# Utils

In [105]:
def epsilon_similarity_graph(X: np.ndarray, sigma=1, epsilon=0):
    """ X (n x d): coordinates of the n data points in R^d.
        sigma (float): width of the kernel
        epsilon (float): threshold
        Return:
        adjacency (n x n ndarray): adjacency matrix of the graph.
    """  
    distance=squareform(pdist(X, 'euclidean'))
    #print(np.mean(distance))
    weights=np.exp(-distance**2/(2*sigma*sigma))
    np.fill_diagonal(weights,0)
    #plt.hist(weights.reshape(weights.shape[0]**2 ,1),bins=100)
    #plt.title("distance treshold")
    #plt.show()
    adjacency=weights
    adjacency[adjacency<epsilon]=0
    return adjacency
"""""
def compute_laplacian(adjacency: np.ndarray, normalize: bool):
    Degrees=np.diag(np.sum(adjacency,1))
    #print(  (np.sum(adjacency,1) == 0).any()   )
    if normalize==False:
        L=Degrees-adjacency
    if normalize==True:
        D_norm=np.diag(np.sum(adjacency,1)**(-1/2))
        L=D_norm@(Degrees-adjacency)@D_norm
    return L
"""""
    


def compute_laplacian(adjacency: np.ndarray, normalize: bool):
    """ Return:
        L (n x n ndarray): combinatorial or symmetric normalized Laplacian.
    """
    D = np.diag(np.sum(adjacency, 1)) # Degree matrix
    combinatorial = D - adjacency
    if normalize:
        D_norm = np.diag(np.clip(np.sum(adjacency, 1), 1, None)**(-1/2))
        return D_norm @ combinatorial @ D_norm
    else:
        return combinatorial

    

def spectral_decomposition(laplacian: np.ndarray):
    lamb, U=sci.linalg.eigh(laplacian)
    sorted_idx = np.argsort(lamb)
    lamb = lamb[sorted_idx]
    U=U[:,sorted_idx]
    return lamb,U


def GFT(signal, U):
    # Your code here
    print(len(signal))
    return U.T @ signal

def iGFT(fourier_coefficients, U):
    # Your code here
    return U @ fourier_coefficients
def ideal_graph_filter(x, spectral_response, U):
    """Return a filtered signal."""
    # Your code here
    # spectral_response is in the fourier domain
    # x is in the graph domain
    # need to converge x to the fourier domain 
    # finally convert all back to the graph domain
    return iGFT(GFT(x,U) * spectral_response, U)

def pred_iteration(A,iters, x, n, filtered_x_lp):
    f1_scores =[]
    for i in iters:
        test_idx = np.random.choice(np.arange(len(x)),n,replace = False)
        # masking some winner
        x[test_idx]=0
        #prepare ground truth
        truth = (df_merged_total["winner"][test_idx]).values

        pred = []
        for i in test_idx:
            l = np.where(A[i] !=0)[0]  # searching neigbhours
            tmp = 0
            for j in l:
                
                tmp += filtered_x_lp[j]
            pred.append(tmp/len(l))   # compute mean
        # thresholding
        pred_thres = np.array(pred)
        pred_thres [pred_thres >0 ] = 1
        pred_thres [pred_thres <0 ] = -1

        # ground truth
        truth = (df_merged_total["winner"][test_idx]).values

        #right_pred = []
        f1_scores.append(sm.f1_score(truth,pred_thres))
        #for i in range(len(pred_thres)):
        #    if(pred_thres[i] == truth[i]):
        #        right_pred.append(i)
        #accuracy.append( len(right_pred)/len(test_idx))
        
    mean = np.mean(f1_scores)
    var = np.std(f1_scores)
    print("The mean is ",mean)
    print("The variance is ",var)
    return mean,var

# Prepare data

In [63]:
# load the data
df_migrations = pd.read_csv("./NTDS_Data/countyinflow1516.csv" )

In [64]:
# load the data
df_migrations = pd.read_csv("./NTDS_Data/countyinflow1516.csv" )

# keep only summury information of each county
df_migrations = df_migrations[df_migrations['y1_countyname'].str.contains("County Total Migration")]

# create the combined fips county number 
df_migrations['statefips_str'] = df_migrations['y2_statefips'].apply(lambda x : str(x).zfill(2))
df_migrations['countyfips_str'] = df_migrations['y2_countyfips'].apply(lambda x : str(x).zfill(3))
df_migrations['combined_fips'] = df_migrations['statefips_str'].apply(lambda x: x.lstrip('0')) + df_migrations['countyfips_str']

# drop useless information 
df_migrations = df_migrations.drop(columns=["y2_statefips", "y2_countyfips", "y1_statefips", "y1_countyfips", "y1_state", "statefips_str", "countyfips_str"])

# seperate each possible migration into three dataframe 
df_migration_total = df_migrations[df_migrations['y1_countyname'].str.contains("County Total Migration-US and Foreign")]
df_migrations['y1_countyname'] = df_migrations['y1_countyname'].apply(lambda x : x if x.find("County Total Migration-US and Foreign") == -1 else "County Total Migration Both")
df_migration_us = df_migrations[df_migrations['y1_countyname'].str.contains("County Total Migration-US")]
df_migration_for = df_migrations[df_migrations['y1_countyname'].str.contains("County Total Migration-Foreign")]

# drop the name of the column 
df_migration_total = df_migration_total.drop(columns=["y1_countyname"])
df_migration_us = df_migration_us.drop(columns=["y1_countyname"])
df_migration_for = df_migration_for.drop(columns=["y1_countyname"])

# remove nodes where data is undefined undefined data by zero
df_migration_total = df_migration_total[df_migration_total['n1'] != -1]
df_migration_us = df_migration_us[df_migration_us['n1'] != -1]
df_migration_for = df_migration_for[df_migration_for['n1'] != -1]

# convert combined fips to int64
df_migration_total['combined_fips'] = df_migration_total['combined_fips'].astype('int64')
df_migration_us['combined_fips'] = df_migration_us['combined_fips'].astype('int64')
df_migration_for['combined_fips'] = df_migration_for['combined_fips'].astype('int64')

In [65]:
df_presidential_result = pd.read_csv("./NTDS_Data/2016_US_County_Level_Presidential_Results.csv" )
df_presidential_result = df_presidential_result.drop(columns=["Unnamed: 0","votes_dem", "votes_gop", "total_votes", "diff", "per_point_diff", "state_abbr", "county_name"])

In [66]:
# merge the two dataset and drop useless column, add a new column winner 
df_merged_total = pd.merge(df_migration_total, df_presidential_result, on="combined_fips", how='inner')
df_merged_us = pd.merge(df_migration_us, df_presidential_result, on="combined_fips", how='inner')
df_merged_for = pd.merge(df_migration_for, df_presidential_result, on="combined_fips", how='inner')
df_merged_total['difference'] = df_merged_total['per_dem'] - df_merged_total['per_gop']
df_merged_us['difference'] = df_merged_us['per_dem'] - df_merged_total['per_gop']
df_merged_for['difference'] = df_merged_for['per_dem'] - df_merged_total['per_gop']
df_merged_total['winner'] = df_merged_total['difference'].apply(lambda x : -1 if x > 0 else 1)
df_merged_us['winner'] = df_merged_us['difference'].apply(lambda x : -1 if x > 0 else 1)
df_merged_for['winner'] = df_merged_for['difference'].apply(lambda x : -1 if x > 0 else 1)
df_merged_total = df_merged_total.drop(columns=['difference'])
df_merged_us = df_merged_us.drop(columns=['difference'])
df_merged_for = df_merged_for.drop(columns=['difference'])

# Compute Adjacency matrix

### Division by zero in normalized lapacien computation

- some nodes do not have any connection in the graph => create new adjacency matrix A by deleting these nodes

In [93]:
X_total = df_merged_total.drop(columns=['combined_fips', 'per_dem', 'per_gop', 'winner'])
nodes_total = df_merged_total.drop(columns=['n1', 'n2', 'agi', 'per_dem', 'per_gop']).values
X_total['agi'] = (X_total['agi'] - X_total['agi'].mean()) / X_total['agi'].std()
X_total['prop_ret/exempt'] = X_total['n1'] / X_total['n2']
X_total = X_total.drop(columns=['n1', 'n2'])
adjacency_RGB_total = epsilon_similarity_graph(X_total, sigma=0.5284353963018223*0.1, epsilon=0.2)
#plt.spy(adjacency_RGB_total)
#plt.show()

# Randomly choose some points for prediction use

In [94]:
# prepare A and x(signal)
A = adjacency_RGB_total.copy()
rows_to_be_deleted = np.where(np.sum(adjacency_RGB_total,1) == 0)
A = np.delete(A, rows_to_be_deleted,0)
A = np.delete(A, rows_to_be_deleted,1)
y = df_merged_total["winner"]
y = np.delete(np.array(y), rows_to_be_deleted,0) 
X_total = X_total.values
X_total = np.delete(np.array(X_total), rows_to_be_deleted,0)
# compute lamb and U
laplacian = compute_laplacian(A, normalize=True)
lamb, U = spectral_decomposition(laplacian)

0       1
1       1
2       1
3       1
4       1
       ..
2958    1
2959   -1
2960    1
2961    1
2962    1
Name: winner, Length: 2963, dtype: int64
[1 1 1 ... 1 1 1]


In [98]:
# prepare filter
n_nodes = A.shape[0]
ideal_lp = np.ones((n_nodes,)) 
ideal_lp[lamb >= 0.1] = 0   # to tune
# apply filter
x_lp = ideal_graph_filter(y,ideal_lp,U)

2939


In [103]:
len(rows_to_be_deleted[0])

24

In [99]:
iters = np.arange(10)
n = int(len(y)*0.2)

In [100]:
accuracy_mean, accuracy_var = pred_iteration(A,iters, y, n, x_lp)

The mean is  0.9019697101103207
The variance is  0.008234275548126271


# FOR

In [22]:
X_for = df_merged_for.drop(columns=['combined_fips', 'per_dem', 'per_gop', 'winner'])
nodes_for = df_merged_for.drop(columns=['n1', 'n2', 'agi', 'per_dem', 'per_gop']).values
X_for['agi'] = (X_for['agi'] - X_for['agi'].mean()) / X_for['agi'].std()
X_for['prop_ret/exempt'] = X_for['n1'] / X_for['n2']
X_for = X_for.drop(columns=['n1', 'n2'])
adjacency_RGB_for = epsilon_similarity_graph(X_for, sigma=0.6675252605174871*0.1, epsilon=0.5)
#plt.spy(adjacency_RGB_for)
#plt.show()

In [23]:
# prepare A and x(signal)
A_for = adjacency_RGB_for.copy()
rows_to_be_deleted_for = np.where(np.sum(adjacency_RGB_for,1) == 0)
A_for = np.delete(A_for, rows_to_be_deleted_for,0)
A_for = np.delete(A_for, rows_to_be_deleted_for,1)
y_for = df_merged_for["winner"]
y_for = np.delete(np.array(y_for), rows_to_be_deleted_for,0) 

X_for = X_for.values
X_for = np.delete(np.array(X_for), rows_to_be_deleted_for,0) 

# compute lamb and U
laplacian_for = compute_laplacian(A_for, normalize=True)
lamb_for, U_for = spectral_decomposition(laplacian_for)

In [24]:
# prepare filter
ideal_lp_for = np.ones((A_for.shape[0],)) 
ideal_lp_for[lamb_for >= 0.1] = 0   # to tune
# apply filter
x_lp_for = ideal_graph_filter(y_for,ideal_lp_for,U_for)

437


In [25]:
iters_for = np.arange(10)
n_for = int(len(y_for)*0.2)

In [26]:
accuracy_mean_for, accuracy_var_for = pred_iteration(A_for,iters_for, y_for, n_for, x_lp_for)

The mean is  0.8413599105861337
The variance is  0.02580566291784558


# US

In [27]:
X_us = df_merged_us.drop(columns=['combined_fips', 'per_dem', 'per_gop', 'winner'])
nodes_us = df_merged_us.drop(columns=['n1', 'n2', 'agi', 'per_dem', 'per_gop']).values
X_us['agi'] = (X_us['agi'] - X_us['agi'].mean()) / X_us['agi'].std()
X_us['prop_ret/exempt'] = X_us['n1'] / X_us['n2']
X_us = X_us.drop(columns=['n1', 'n2'])
adjacency_RGB_us = epsilon_similarity_graph(X_us, sigma=0.5310405705207334*0.1, epsilon=0.5)
#plt.spy(adjacency_RGB_us)
#plt.show()

In [28]:
# prepare A and x(signal)
A_us = adjacency_RGB_us.copy()
rows_to_be_deleted_us = np.where(np.sum(adjacency_RGB_us,1) == 0)
A_us = np.delete(A_us, rows_to_be_deleted_us,0)
A_us = np.delete(A_us, rows_to_be_deleted_us,1)
y_us = df_merged_us["winner"]
y_us = np.delete(np.array(y_us), rows_to_be_deleted_us,0) 

X_us = X_us.values
X_us = np.delete(np.array(X_us), rows_to_be_deleted_us,0) 
# compute lamb and U
laplacian_us = compute_laplacian(A_us, normalize=True)
lamb_us, U_us = spectral_decomposition(laplacian_us)

In [29]:
# prepare filter
ideal_lp_us = np.ones((A_us.shape[0],)) 
ideal_lp_us[lamb_us >= 0.1] = 0   # to tune
# apply filter
x_lp_us = ideal_graph_filter(y_us,ideal_lp_us,U_us)

2921


In [30]:
iters_us = np.arange(10)
n_us = int(len(y_us)*0.2)

In [31]:
accuracy_mean_us, accuracy_var_us = pred_iteration(A_us,iters_us, y_us, n_us, x_lp_us)

The mean is  0.9028313817355549
The variance is  0.0052804173512027595


In [ ]:
# comparer avec la version sans filtrage pour avoir combien on améliore

# GCN

In [40]:
import time

import networkx as nx
from sklearn.linear_model import LogisticRegression

import torch
import torch.nn as nn
import torch.nn.functional as F

import dgl.function as fn
from dgl import DGLGraph
from dgl.data.citation_graph import load_cora

np.random.seed(0)
torch.manual_seed(1)

In [41]:
class LaplacianPolynomial(nn.Module):
    def __init__(self,
                 in_feats: int,
                 out_feats: int,
                 k: int,
                 dropout_prob: float,
                 norm=True):
        super().__init__()
        self._in_feats = in_feats
        self._out_feats = out_feats
        self._k = k
        self._norm = norm
        # Contains the weights learned by the Laplacian polynomial
        self.pol_weights = nn.Parameter(torch.Tensor(self._k + 1))
        # Contains the weights learned by the logistic regression (without bias)
        self.logr_weights = nn.Parameter(torch.Tensor(in_feats, out_feats))
        self.dropout = nn.Dropout(p=dropout_prob)
        self.reset_parameters()

    def reset_parameters(self):
        """Reinitialize learnable parameters."""
        torch.manual_seed(0)
        torch.nn.init.xavier_uniform_(self.logr_weights, gain=0.01)
        torch.nn.init.normal_(self.pol_weights, mean=0.0, std=1e-3)

    def forward(self, graph, feat):
        r"""Compute graph convolution.

        Notes
        -----
        * Input shape: :math:`(N, *, \text{in_feats})` where * means any number of additional
          dimensions, :math:`N` is the number of nodes.
        * Output shape: :math:`(N, *, \text{out_feats})` where all but the last dimension are
          the same shape as the input.

        Parameters
        ----------
        graph (DGLGraph) : The graph.
        feat (torch.Tensor): The input feature

        Returns
        -------
        (torch.Tensor) The output feature
        """
        feat = self.dropout(feat)
        graph = graph.local_var()
        
        # D^(-1/2)
        norm = torch.pow(graph.in_degrees().float().clamp(min=1), -0.5)
        shp = norm.shape + (1,) * (feat.dim() - 1)
        norm = torch.reshape(norm, shp)

        # mult W first to reduce the feature size for aggregation.
        feat = torch.matmul(feat, self.logr_weights)

        result = self.pol_weights[0] * feat.clone()

        for i in range(1, self._k + 1):
            old_feat = feat.clone()
            if self._norm:
                feat = feat * norm
            graph.ndata['h'] = feat
            # Feat is not modified in place
            graph.update_all(fn.copy_src(src='h', out='m'),
                             fn.sum(msg='m', out='h'))
            if self._norm:
                graph.ndata['h'] = graph.ndata['h'] * norm

            feat = old_feat - graph.ndata['h']
            result += self.pol_weights[i] * feat

        return result

    def extra_repr(self):
        """Set the extra representation of the module,
        which will come into effect when printing the model.
        """
        summary = 'in={_in_feats}, out={_out_feats}'
        summary += ', normalization={_norm}'
        return summary.format(**self.__dict__)

In [42]:
def train(model, g, features, labels, loss_fcn, train_mask, optimizer):
    model.train()  # Activate dropout
    
    logits = model(g, features)
    loss = loss_fcn(logits[train_mask], labels[train_mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss

def evaluate(model, g, features, labels, mask):
    model.eval()  # Deactivate dropout
    with torch.no_grad():
        logits = model(g, features)[mask]  # only compute the evaluation set
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)
    
def polynomial_graph_filter(coeff: np.array, laplacian: np.ndarray):
    """ Return the laplacian polynomial with coefficients 'coeff'. """
    # Your code here
    res = np.zeros_like(laplacian)
    for i in range (len(coeff)):
        res += coeff[i] * np.linalg.matrix_power(laplacian, i)
    return res

def polynomial_graph_filter_response(coeff: np.array, lam: np.ndarray):
    """ Return an array of the same shape as lam.
        response[i] is the spectral response at frequency lam[i]. """
    # Your code here
    res = np.zeros((len(coeff),len(lam)))
    for i in range(len(coeff)):
        res[i] = coeff[i] * (lam**i)
    res = np.sum(res,axis =0)
    return res

In [54]:
def apply_gcn(iters,X,y,A,laplacian,lamb,U):
    f1_scores = []
    print("Computing")
    for i in range(iters):
        if( i != 0 and i%(iters*0.1) == 0):
            print(str(int(i*100/iters))+" %")
        # Some basic settings
        features = torch.FloatTensor(X)
        labels = torch.LongTensor(y) 
        in_feats = features.shape[1]  # 2
        n_classes = 2
        n_edges = int(A.sum() // 2)
        pol_order = 3
        lr = 0.2
        weight_decay = 5e-6
        n_epochs = 500
        p_dropout = 0.8

        # prepare for masking
        n_points = X.shape[0]
        indices = np.arange(n_points)
        np.random.shuffle(indices)
        split_t = int(n_points*0.2)
        test_idx = indices[:split_t]
        train_idx = indices[split_t:]
        train_mask = np.zeros(n_points)
        train_mask[train_idx] = 1
        test_mask = np.zeros(n_points)
        test_mask[test_idx] = 1
        graph = nx.from_numpy_matrix(A)
        adjacency = np.asarray(nx.to_numpy_matrix(graph))
        n_nodes = adjacency.shape[0]
        lam_max = np.max(lamb)


        graph = DGLGraph(graph)
        model = LaplacianPolynomial(in_feats, n_classes, pol_order, p_dropout)

        loss_fcn = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(),
                                     lr=lr,
                                     weight_decay=weight_decay)

        dur = []
        for epoch in range(50):
            loss = train(model, graph, features, labels, loss_fcn, train_mask, optimizer)

        coeff_gcn =  model.pol_weights.detach().numpy()
        graph_gcn_filter = polynomial_graph_filter(coeff_gcn, laplacian)
        features_gcn = graph_gcn_filter @ features.numpy()
        train_mask = torch.BoolTensor(train_mask) 
        test_mask = torch.BoolTensor(test_mask)
        train_features_gcn = features_gcn[train_mask,:]
        train_labels = labels[train_mask]
        test_features_gcn = features_gcn[test_mask,:]
        test_labels = labels[test_mask]
        print(train_labels)
        model =  LogisticRegression(C=1000,penalty = 'l2',solver='liblinear', multi_class='auto',max_iter = 2000)
        model.fit(train_features_gcn, train_labels)
        
        test_pred = model.predict(test_features_gcn)
        #test_acc =  model.score(test_features_gcn,test_labels)  
        #accuracy.append(test_acc)
        print(test_pred)
        print("---")
        f1_scores.append(sm.f1_score(test_labels,test_pred))
        #for i in range(len(pred_thres)):
        #    if(pred_thres[i] == truth[i]):
        #        right_pred.append(i)
        #accuracy.append( len(right_pred)/len(test_idx))
        
    print("100 %")
    mean = np.mean(f1_scores)
    var = np.std(f1_scores)
    print("The mean is ",mean)
    print("The variance is ",var)
    return mean,var        

In [56]:
y_for

array([ 0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,
        1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,
        1,  0,  0,  0,  0,  0,  0,  0,  0, -1, -1,  0,  0,  0,  0,  1,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
       -1,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  1,
        1,  0,  1,  0,  0

In [79]:
mean_us,var_us = apply_gcn(20,X_us,y_us,A_us,laplacian_us,lamb_us,U_us)

0%
10%
20%
30%
40%
50%
60%
70%
80%
90%
The mean is  0.8863869863013699
The variance is  0.009690555491542241


(0.8863869863013699, 0.009690555491542241)

In [80]:
mean_total,var_total = apply_gcn(20,X_total,y,A,laplacian,lamb,U)

0%
10%
20%
30%
40%
50%
60%
70%
80%
90%
The mean is  0.910817717206133
The variance is  0.010885247108108901


(0.910817717206133, 0.010885247108108901)

# Result 
accuracy: `mean-var`

            Total      For        Us
            
`GCN :     0.91-0.01  0.71-0.04  0.88-0.01`

`Fourier : 0.82-0.01  0.74-0.05  0.82-0.01`




In [ ]:
# overfit? => comparer les ecarts de scores entre train set et test set
# ne petmettre pas de trouver les patterns


# metric when data note balanced: F1 score